In [10]:
import tensorflow as tf
import numpy as np
import os
import scipy.io as spio
from matplotlib import pyplot as plt
from scipy.misc import imread
from create_annotations import from_xml

#### Obs: If you only have one of the datasets (does not matter which one), just run all the notebook's cells and it will work just fine.

In [11]:
# ICDAR POD 2017
base_dataset_dir_voc = '/Users/ankur/Projects/cosmos/Train'
images_folder_name_voc = "Image/"
annotations_folder_name_voc = "Annotations/"
images_dir_voc = os.path.join(base_dataset_dir_voc, images_folder_name_voc)
annotations_dir_voc = os.path.join(base_dataset_dir_voc, annotations_folder_name_voc)

In [12]:
def get_files_list(base_dataset_dir, images_folder_name, annotations_folder_name, filename):
    images_dir = os.path.join(base_dataset_dir, images_folder_name)
    annotations_dir = os.path.join(base_dataset_dir, annotations_folder_name)

    file = open(filename, 'r')
    images_filename_list = [line for line in file]
    return images_filename_list

In [13]:
images_filename_list = get_files_list(base_dataset_dir_voc, images_folder_name_voc, annotations_folder_name_voc, "image_names.txt")
print("Total number of training images:", len(images_filename_list))

Total number of training images: 1600


In [14]:
# shuffle array and separate 10% to validation
np.random.shuffle(images_filename_list)
val_images_filename_list = images_filename_list[:int(0.10*len(images_filename_list))]
train_images_filename_list = images_filename_list[int(0.10*len(images_filename_list)):]

In [15]:
print("train set size:", len(train_images_filename_list))
print("val set size:", len(val_images_filename_list))

train set size: 1440
val set size: 160


In [16]:
TRAIN_DATASET_DIR="./tfrecords/"
if not os.path.exists(TRAIN_DATASET_DIR):
    os.mkdir(TRAIN_DATASET_DIR)
    
TRAIN_FILE = 'train.tfrecords'
VALIDATION_FILE = 'validation.tfrecords'
TEST_FILE = 'test.tfrecords'
train_writer = tf.python_io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,TRAIN_FILE))
val_writer = tf.python_io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,VALIDATION_FILE))
#test_writer = tf.python_io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,TEST_FILE))

In [17]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [18]:
def read_annotation_from_mat_file(annotations_dir, image_name):
    annotations_path = os.path.join(annotations_dir, (image_name.strip() + ".mat"))
    mat = spio.loadmat(annotations_path)
    img = mat['GTcls']['Segmentation'][0][0]
    return img

In [19]:
def create_tfrecord_dataset(filename_list, writer):

    # create training tfrecord
    read_imgs_counter = 0
    for i, image_name in enumerate(filename_list):

        try:
            # read from Pascal VOC path
            image_np = imread(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))
        except FileNotFoundError:
            print("File:",image_name.strip(),"not found.")
            continue
        

        read_imgs_counter += 1
        image_h = image_np.shape[0]
        image_w = image_np.shape[1]
        
        
        try:
            xml_f = os.path.join(annotations_dir_voc, image_name.strip() + ".xml")
            annotation_np = from_xml(xml_f, image_h, image_w)
            # [['105', '279'], ['307', '279'], ['105', '298'], ['307', '298']]
        except FileNotFoundError:
            print("File:",image_name.strip(),"not found.")
            continue
#        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 8))
#        ax1.imshow(image_np)
#        ax2.imshow(annotation_np)
#        plt.savefig('images/{}.png'.format(image_name))
        img_raw = image_np.tostring()
        annotation_raw = annotation_np.tostring()
        if i == 0:
            print(image_np.dtype)
            print(image_np.shape)
            print(len(img_raw))
            print(annotation_np.dtype)
            print(annotation_np.shape)
            print(len(annotation_raw))

        example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(image_h),
                'width': _int64_feature(image_w),
                'image_raw': _bytes_feature(img_raw),
                'annotation_raw': _bytes_feature(annotation_raw)}))

        writer.write(example.SerializeToString())
    
    print("End of TfRecord. Total of image written:", read_imgs_counter)
    writer.close()

In [20]:
# create training dataset
create_tfrecord_dataset(train_images_filename_list, train_writer)

/Users/ankur/.virtualenvs/deeplab3/lib/python3.5/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if __name__ == '__main__':


uint8
(1123, 749, 3)
2523381
uint8
(1123, 749)
841127
End of TfRecord. Total of image written: 1440


In [21]:
# create validation dataset
create_tfrecord_dataset(val_images_filename_list, val_writer)

uint8
(1459, 1031, 3)
4512687
uint8
(1459, 1031)
1504229


/Users/ankur/.virtualenvs/deeplab3/lib/python3.5/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if __name__ == '__main__':


End of TfRecord. Total of image written: 160


In [9]:
create_tfrecord_dataset(images_filename_list, test_writer)

/Users/ankur/.virtualenvs/deeplab3/lib/python3.5/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if __name__ == '__main__':


uint8
(1123, 749, 3)
2523381
uint8
(1123, 749)
841127
End of TfRecord. Total of image written: 817
